<div class=title>
<strong><center><font size="3">DAY 23 - IYKRA</font></center>
<hr>
<center><font size="5"><strong>Data Science Bootcamp</strong></font></center>
<center><font size="4"><strong>Applied Machine Learning: Model</strong></font></center>
<hr>
<p style="text-align:center">Author</p>
<center>Joshua Effendi and Gilang Mahardika</center>
<p style="text-align:center">Date:</p>
<center>20 November 2019</center></strong>
</div>

<div class=title1>
    <b><font size='3'>Load Packages</font></b>
</div>

In [31]:
import numpy as np # math calculation
import pandas as pd # data wrangling
import matplotlib.pyplot as plt # plot visualization
import seaborn as sns # visualization
from sklearn.metrics import auc, confusion_matrix
from sklearn import metrics
import pickle

import warnings
warnings.filterwarnings("ignore")

from IPython.core.display import HTML

def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

<div class=title1>
    <b><font size='3'>Load Dataset</font></b>
</div>
<div class=title2>
    <b><font size='2'>Data yang digunakan adalah data german_credit.csv untuk melakukan analisis regresi</font></b>
</div>

In [23]:
data = pd.read_csv("german_credit.csv")

In [24]:
data.head()

,default,account_check_status,duration_in_month,credit_history,purpose,credit_amount,savings,present_emp_since,installment_as_income_perc,personal_status_sex,...,present_res_since,property,age,other_installment_plans,housing,credits_this_bank,job,people_under_maintenance,telephone,foreign_worker
0,0,< 0 DM,6,critical account/ other credits existing (not ...,domestic appliances,1169,unknown/ no savings account,.. >= 7 years,4,male : single,...,4,real estate,67,none,own,2,skilled employee / official,1,"yes, registered under the customers name",yes
1,1,0 <= ... < 200 DM,48,existing credits paid back duly till now,domestic appliances,5951,... < 100 DM,1 <= ... < 4 years,2,female : divorced/separated/married,...,2,real estate,22,none,own,1,skilled employee / official,1,none,yes
2,0,no checking account,12,critical account/ other credits existing (not ...,(vacation - does not exist?),2096,... < 100 DM,4 <= ... < 7 years,2,male : single,...,3,real estate,49,none,own,1,unskilled - resident,2,none,yes
3,0,< 0 DM,42,existing credits paid back duly till now,radio/television,7882,... < 100 DM,4 <= ... < 7 years,2,male : single,...,4,if not A121 : building society savings agreeme...,45,none,for free,1,skilled employee / official,2,none,yes
4,1,< 0 DM,24,delay in paying off in the past,car (new),4870,... < 100 DM,1 <= ... < 4 years,3,male : single,...,4,unknown / no property,53,none,for free,2,skilled employee / official,2,none,yes


In [25]:
print(data['personal_status_sex'].value_counts())
print('\n')
print(data['default'].value_counts())
print('\n')
print(data['job'].value_counts())

male : single                          548
female : divorced/separated/married    310
male : married/widowed                  92
male : divorced/separated               50
Name: personal_status_sex, dtype: int64


0    700
1    300
Name: default, dtype: int64


skilled employee / official                                      630
unskilled - resident                                             200
management/ self-employed/ highly qualified employee/ officer    148
unemployed/ unskilled - non-resident                              22
Name: job, dtype: int64


<div class=title1>
    <b><font size='3'>Validation Data</font></b>
</div>

In [26]:
validation = data.sample(200, random_state = 1234)
validation.head()

,default,account_check_status,duration_in_month,credit_history,purpose,credit_amount,savings,present_emp_since,installment_as_income_perc,personal_status_sex,...,present_res_since,property,age,other_installment_plans,housing,credits_this_bank,job,people_under_maintenance,telephone,foreign_worker
681,0,no checking account,12,existing credits paid back duly till now,domestic appliances,2279,unknown/ no savings account,1 <= ... < 4 years,4,male : single,...,4,unknown / no property,37,none,for free,1,skilled employee / official,1,"yes, registered under the customers name",yes
990,0,no checking account,12,critical account/ other credits existing (not ...,(vacation - does not exist?),3565,unknown/ no savings account,... < 1 year,2,male : single,...,1,if not A121 : building society savings agreeme...,37,none,own,2,unskilled - resident,2,none,yes
155,1,< 0 DM,12,existing credits paid back duly till now,radio/television,1282,... < 100 DM,1 <= ... < 4 years,2,female : divorced/separated/married,...,4,"if not A121/A122 : car or other, not in attrib...",20,none,rent,1,skilled employee / official,1,none,yes
768,0,0 <= ... < 200 DM,12,critical account/ other credits existing (not ...,radio/television,3617,... < 100 DM,.. >= 7 years,1,male : single,...,4,"if not A121/A122 : car or other, not in attrib...",28,none,rent,3,skilled employee / official,1,"yes, registered under the customers name",yes
438,0,< 0 DM,42,critical account/ other credits existing (not ...,education,3394,... < 100 DM,unemployed,4,male : single,...,4,"if not A121/A122 : car or other, not in attrib...",65,none,own,2,unemployed/ unskilled - non-resident,1,none,yes


In [28]:
validation = validation.drop(columns='default').reset_index(drop=True)

In [30]:
validation.to_json('validation.json')

In [ ]:
#Drop validation data from raw data
data = data.drop(validation.index)

<div class=title1>
    <b><font size='3'>Feature Engineering</font></b>
</div>

In [ ]:
def age_group(age):
    if age < 35:
        return 'Productive'
    if age < 50:
        return 'Mature'
    else:
        return 'Old'

In [ ]:
data['age_group'] = data['age'].apply(age_group)

In [ ]:
data.head()

In [ ]:
validation['age_group'] = validation['age'].apply(age_group)

In [ ]:
validation.head()

<div class=title1>
    <b><font size='3'>Train-Test Split</font></b>
</div>

In [ ]:
from sklearn.preprocessing import StandardScaler

xgb_data = data.copy()
target = xgb_data['default']
temp = xgb_data.drop(['default'], axis = 1)

In [ ]:
dummy = temp.select_dtypes(exclude = ['int', 'int64', 'float64'])
integer = temp.select_dtypes(include = ['int', 'int64', 'float64'])

In [ ]:
dummy2 = pd.get_dummies(dummy, drop_first = True)

In [ ]:
xgb_data = pd.concat([integer, dummy2, target], axis = 1)

In [ ]:
X = xgb_data.iloc[:, 0:len(xgb_data.columns)-1]
y = xgb_data.iloc[:, len(xgb_data.columns)-1]

In [ ]:
valid_data = validation.copy()
valid_target = valid_data['default']
valid_temp = valid_data.drop(['default'], axis = 1)

In [ ]:
valid_dummy = valid_temp.select_dtypes(exclude = ['int', 'int64', 'float64'])
valid_integer = valid_temp.select_dtypes(include = ['int', 'int64', 'float64'])

In [ ]:
valid_dummy2 = pd.get_dummies(valid_dummy, drop_first = True)
valid_data = pd.concat([valid_integer, valid_dummy2, valid_target], axis = 1)

In [ ]:
X_valid = valid_data.iloc[:, 0:len(valid_data.columns)-1]
y_valid = valid_data.iloc[:, len(valid_data.columns)-1]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1234)

In [ ]:
SC = StandardScaler()
X_train = SC.fit_transform(X_train)
X_test = SC.fit_transform(X_test)

<div class=title1>
    <b><font size='3'>Oversampling</font></b>
</div>

In [ ]:
from imblearn.over_sampling import SMOTE

smt = SMOTE(random_state= 1234)
X_train, y_train = smt.fit_sample(X_train, y_train)

<div class=title1>
    <b><font size='3'>XGBoost Model</font></b>
</div>

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
#print('XGBoost v',xgb.__version__)

# fit, train and cross validate Decision Tree with training and test data 
def xgbclf(params, X_train, y_train,X_test, y_test):
  
    eval_set=[(X_train, y_train), (X_test, y_test)]
    
    model = XGBClassifier(**params).\
      fit(X_train, y_train, eval_set=eval_set, \
                  eval_metric='auc', early_stopping_rounds = 100, verbose=100)
        
    #print(model.best_ntree_limit)

    model.set_params(**{'n_estimators': model.best_ntree_limit})
    model.fit(X_train, y_train)
    #print(model,'\n')
    
    # Predict target variables y for test data
    y_pred = model.predict(X_test, ntree_limit=model.best_ntree_limit) #model.best_iteration
    #print(y_pred)
   
    # Get Cross Validation and Confusion matrix
    #get_eval(model, X_train, y_train)
    #get_eval2(model, X_train, y_train,X_test, y_test)
    
    # Create and print confusion matrix    
    abclf_cm = confusion_matrix(y_test,y_pred)
    print(abclf_cm)
    
    #y_pred = model.predict(X_test)
    print (metrics.classification_report(y_test,y_pred) )
    print ('\n')
    print ("Model Final Generalization Accuracy: %.6f" %metrics.accuracy_score(y_test,y_pred) )
    
    # Predict probabilities target variables y for test data
    y_pred_proba = model.predict_proba(X_test, ntree_limit=model.best_ntree_limit)[:,1] #model.best_iteration
    get_roc (y_test,y_pred_proba)
    return model

def plot_featureImportance(model, keys):
  importances = model.feature_importances_

  importance_frame = pd.DataFrame({'Importance': list(importances), 'Feature': list(keys)})
  importance_frame.sort_values(by = 'Importance', inplace = True)
  importance_frame.tail(10).plot(kind = 'barh', x = 'Feature', figsize = (8,8), color = 'orange')

<div class=title1>
    <b><font size='3'>Model Accuracy</font></b>
</div>

In [ ]:
def get_roc (y_test,y_pred):
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    fpr, tpr, _ = metrics.roc_curve(y_test, y_pred)
    roc_auc = metrics.auc(fpr, tpr)
    #Plot of a ROC curve
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
             label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="upper left")
    plt.show()
    return

<div class=title1>
    <b><font size='3'>XGBoost Model Test</font></b>
</div>

In [ ]:
params={}

params1={
    'n_estimators':3000,
    'objective': 'binary:logistic',
    'learning_rate': 0.05,
    'gamma':0.1,
    'subsample':0.8,
    'colsample_bytree':0.3,
    'min_child_weight':3,
    'max_depth':3,
    #'seed':1024,
    'n_jobs' : -1
}

params2={
    'n_estimators':3000,
    'objective': 'binary:logistic',
    'learning_rate': 0.005,
    #'gamma':0.01,
    'subsample':0.555,
    'colsample_bytree':0.7,
    'min_child_weight':3,
    'max_depth':8,
    #'seed':1024,
    'n_jobs' : -1
}

xgbclf(params2, X_train, y_train, X_test, y_test)

In [ ]:
eval_set=[(X_train, y_train), (X_test, y_test)]
    
model = XGBClassifier(**params).\
fit(X_train, y_train, eval_set=eval_set, \
eval_metric='auc', early_stopping_rounds = 100, verbose=100)
        
    #print(model.best_ntree_limit)
model.set_params(**{'n_estimators': model.best_ntree_limit})
model.fit(X_train, y_train)

<div class=title1>
    <b><font size='3'>Dump Model on Pickle</font></b>
</div>

In [ ]:
pickle.dump(model, open('model_xgb.pkl','wb'))